In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os
import scipy.signal
from utils import encode, ofdm, channels, audio, decode, preamble, file_preparation
import sounddevice as sd
cwd = os.getcwd()

In [ ]:
DFT_LENGTH = 4096
CP_LENGTH = 512
OFDM_LENGTH = 4608
silence_duration = 1
chirp_duration = 1
fs = 48000

low_freq = 1000
high_freq = 10000
double_chirp = False
bits_per_symbol = 2
symbol_per_frame = 128
frame_count = 1.75

In [ ]:
new = False

In [ ]:
# Generate a bit array as data to transmit


spb = ofdm.subcarriers_per_block(fs,DFT_LENGTH,low_freq,high_freq)

if new:
    np.random.seed(8)
    info = np.random.randint(2, size=int(frame_count * 2 * spb * symbol_per_frame))
    np.savetxt('info', info, fmt='%i')
else:
    info = file_preparation.file_encode(filesize=39955,filename='Louis.txt', txt_name='Louis_front.txt')
    np.savetxt('info', info, fmt='%i')
    info_list = list(map(str,list(info)))
    info_string = ''.join(info_list)
    f = open("input.txt", mode='w')
    f.write(info_string)
    f.close()

# Load a saved bit array and save its encoded version
info = np.loadtxt('info', dtype=np.int32)
# encoded = encode.ldpc_encode(info)
encoded = info
np.savetxt('encoded', encoded, fmt='%i')

In [ ]:
symbols = encode.qpsk_encode(encoded)

# shifted = ofdm.subcarrier_shift_gaussian(symbols, DFT_LENGTH, fs, low_freq, high_freq, 0.01, bits_per_symbol, constellation='qpsk')
shifted = ofdm.subcarrier_shift_ofdm(symbols, DFT_LENGTH, fs, low_freq, high_freq)
ofdm_data = ofdm.symbols_to_ofdm(shifted, DFT_LENGTH, CP_LENGTH)

In [ ]:
#generate known ofdm symbols
repeat_time = 4
known_ofdm_data = preamble.load_known_ofdm(CP_LENGTH,repeat_time)

scale = max(np.real(known_ofdm_data)) / max(np.real(ofdm_data))

reshape_ofdm = np.reshape(ofdm_data*scale,(-1,OFDM_LENGTH))

#chirp between frame
chirp = preamble.generate_chirp(chirp_duration, fs, low=low_freq, high=high_freq, silence_duration=0, double=double_chirp)
peak = max(np.real(known_ofdm_data)) / max(np.real(chirp))

chirp_inter = peak * chirp

i=0
frame_audio = np.array([])
while (i+128) < reshape_ofdm.shape[0]:
    ofdm_block = reshape_ofdm[i:i+128,:].flatten()
    frame = preamble.frame_assemble(chirp_inter, known_ofdm_data, ofdm_block)
    frame_audio = np.append(frame_audio,frame)
    i += 128

ofdm_block = reshape_ofdm[i:,:].flatten()
frame = preamble.frame_assemble(chirp_inter, known_ofdm_data, ofdm_block)
frame_audio = np.append(frame_audio,frame)

start_audio = peak * preamble.transmission_start(fs,low_freq,high_freq,silence_duration)
end_audio = peak * preamble.transmission_end(fs,low_freq,high_freq,silence_duration)

output_audio = np.concatenate((start_audio,frame_audio,end_audio))
plt.plot(output_audio)

In [ ]:
#save and play
audio.signal_to_wav(np.real(output_audio), fs, 'new_tester_1.75_frames', 'audio_files')
plt.plot(np.real(output_audio))
plt.show()

In [ ]:
sd.play(output_audio,fs,blocking=True)